In [225]:
import capstone
import shutil
import struct

In [226]:
HEADER_SIZE = 64 
E_IDENT = 0
E_TYPE = 0
E_MACHINE = 0
E_VERSION = 0
E_ENTRY = 0
E_PHOFF = 0
E_SHOFF = 0
E_FLAGS = 0
E_EHSIZE = 0
E_PHENTSIZE = 0
E_PHNUM = 0
E_SHENTSIZE = 0
E_SHNUM = 0
E_SHSTRNDX = 0

In [228]:
def print_elf_header(header_tuple, verbose = False):
    global E_IDENT 
    global E_TYPE 
    global E_MACHINE 
    global E_VERSION 
    global E_ENTRY 
    global E_PHOFF 
    global E_SHOFF 
    global E_FLAGS 
    global E_EHSIZE 
    global E_PHENTSIZE 
    global E_PHNUM 
    global E_SHENTSIZE 
    global E_SHNUM 
    global E_SHSTRNDX 

    E_IDENT = header_tuple[0]  
    E_TYPE = header_tuple[1]
    E_MACHINE = header_tuple[2]
    E_VERSION = header_tuple[3]
    E_ENTRY = header_tuple[4]
    E_PHOFF = header_tuple[5]
    E_SHOFF = header_tuple[6]
    E_FLAGS = header_tuple[7]
    E_EHSIZE = header_tuple[8]
    E_PHENTSIZE = header_tuple[9]
    E_PHNUM = header_tuple[10]
    E_SHENTSIZE = header_tuple[11]
    E_SHNUM = header_tuple[12]
    E_SHSTRNDX = header_tuple[13]

    if verbose:
        print("e_ident:", E_IDENT)
        print("e_type:", E_TYPE)
        print("e_machine:", E_MACHINE)
        print("e_version:", E_VERSION)
        print("e_entry:", hex(E_ENTRY))
        print("e_phoff:", hex(E_PHOFF))
        print("e_shoff:", hex(E_SHOFF))
        print("e_flags:", E_FLAGS)
        print("e_ehsize:", E_EHSIZE)
        print("e_phentsize:", E_PHENTSIZE)
        print("e_phnum:", E_PHNUM)
        print("e_shentsize:", E_SHENTSIZE)
        print("e_shnum:", E_SHNUM)
        print("e_shstrndx:", E_SHSTRNDX)

In [ ]:
def read_elf_header(file_path):
    with open(file_path, 'rb') as f:
        elf_data = f.read()

    if elf_data[:4] != b'\x7fELF':
        raise ValueError("Not a valid ELF file.")

    elf_header_format = (
        # e_ident (16 bytes), e_type (2 bytes), e_machine (2 bytes), e_version (4 bytes)
        '< 16s H H I' +
        # e_entry (8 bytes), e_phoff (8 bytes), e_shoff (8 bytes), e_flags (4 bytes)
        'Q Q Q I' +
        # e_ehsize (2 bytes), e_phentsize (2 bytes), e_phnum (2 bytes), e_shentsize (2 bytes)
        'H H H H' +
        # e_shnum (2 bytes), e_shstrndx (2 bytes)
        'H H'
    )

    amd_machine = 0x003e

    unpacked_data = list(struct.unpack(elf_header_format, elf_data[:HEADER_SIZE]))
    unpacked_data[2] = amd_machine

    print_elf_header(unpacked_data)

    packed_data = struct.pack(elf_header_format, *unpacked_data)

    with open(file_path, 'wb') as f:
        f.write(packed_data)
        f.write(elf_data[HEADER_SIZE:])

    return unpacked_data[6], unpacked_data[11]


In [ ]:
def disassemble_code(code_section):
    # Initialize Capstone with the AArch64 architecture
    md = capstone.Cs(capstone.CS_ARCH_ARM64, capstone.CS_MODE_ARM)

    # Disassemble the code section
    instructions = md.disasm(code_section, 0x1000)  # 0x1000 is an arbitrary address

    for insn in instructions:
        print(f"0x{insn.address:x}:\t{insn.mnemonic}\t{insn.op_str}")

In [230]:
def find_code_sections(file_path):
    with open(file_path, 'rb') as f:
        elf_data = f.read()

    code_section = None
    for i in range(e_shnum):
        sh_offset = struct.unpack('<Q', elf_data[e_shoff + i * e_shentsize + 0x18:e_shoff + i * e_shentsize + 0x20])[0]
        sh_size = struct.unpack('<Q', elf_data[e_shoff + i * e_shentsize + 0x20:e_shoff + i * e_shentsize + 0x28])[0]
        sh_type = struct.unpack('<I', elf_data[e_shoff + i * e_shentsize + 0x04:e_shoff + i * e_shentsize + 0x08])[0]
        
        if sh_type == 1:  # SHT_PROGBITS is the type for code sections
            code_section = elf_data[sh_offset:sh_offset + sh_size]
            break

    if not code_section:
        raise ValueError("No code section found in the ELF file.")
    
    return code_section

In [231]:
input = 'test-aarch64.o'  
output = 'out.o'
good_output = 'test-aarch64-x64.o'  

shutil.copy(input, output)

e_shoff, e_shentsize = read_elf_header(output)
print(E_MACHINE)
# find_code_sectons(output, e_shoff, e_shentsize)


62
